# Legal Document Processing, Embedding, and Search
This notebook demonstrates how to process Vietnamese legal documents, create embeddings, and search for relevant information.

In [28]:
import json
import torch
import numpy as np
import re
from typing import List, Dict, Any, Optional
from dataclasses import dataclass, field, asdict
from llama_index.core import Document
from llama_index.core import StorageContext
from llama_index.core import VectorStoreIndex
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.vector_stores.weaviate import WeaviateVectorStore
from pyvi import ViTokenizer
import weaviate
from weaviate.classes.init import Auth
from sentence_transformers import SentenceTransformer

## Legal Document Processing Classes
These classes handle the specialized processing of legal documents, including chunking, metadata extraction, and embedding creation.

In [29]:
@dataclass
class LegalReference:
    """Lưu thông tin về tham chiếu pháp lý."""
    article: Optional[str] = None
    paragraph: Optional[str] = None
    point: Optional[str] = None
    is_current_article: bool = False
    raw_text: str = ""

@dataclass
class ChunkMetadata:
    """Metadata cho chunk pháp lý."""
    article_id: str
    title: str = ""
    type: str = "article"  # article, article_part, paragraph
    paragraphs: List[str] = field(default_factory=list)
    points: List[str] = field(default_factory=list)
    references: List[LegalReference] = field(default_factory=list)

@dataclass
class LegalChunk:
    """Một đoạn văn bản pháp lý với metadata."""
    text: str
    metadata: ChunkMetadata
    embedding: Optional[np.ndarray] = None

class LegalDocument:
    """Xử lý tài liệu pháp lý để chunking và embedding."""
    
    def __init__(self, max_chunk_tokens=500, overlap_tokens=100):
        self.max_chunk_tokens = max_chunk_tokens
        self.overlap_tokens = overlap_tokens
        self.reference_patterns = {
            "other_article": re.compile(r'(điểm|khoản) ([a-z]|\d+)(?:,\s*(điểm|khoản) ([a-z]|\d+))* (Điều|khoản) (\d+)', re.IGNORECASE),
            "same_article": re.compile(r'(điểm|khoản) ([a-z]|\d+)(?:,\s*(điểm|khoản) ([a-z]|\d+))* (Điều này)', re.IGNORECASE),
            "article_only": re.compile(r'Điều (\d+)', re.IGNORECASE)
        }
    
    def process(self, text: str) -> List[LegalChunk]:
        """Xử lý văn bản pháp lý thành các chunk."""
        # Phân tách văn bản thành các điều
        articles = re.split(r'(Điều \d+\.)', text)
        chunks = []
        
        for i in range(1, len(articles), 2):
            article_title = articles[i].strip()
            article_content = articles[i+1].strip() if i+1 < len(articles) else ""
            
            # Trích xuất số điều
            article_id = re.search(r'Điều (\d+)\.', article_title).group(1)
            
            # Trích xuất tiêu đề điều
            title_match = re.search(r'^([^0-9\.\n]*)(?=\d+\.|$)', article_content)
            title = title_match.group(1).strip() if title_match else ""
            
            # Quyết định phương pháp phân chunk
            article_chunks = self._process_article(
                article_title, article_id, title, article_content
            )
            chunks.extend(article_chunks)
        
        return chunks
    
    def _process_article(self, article_title, article_id, title, article_content) -> List[LegalChunk]:
        """Xử lý một điều thành các chunk."""
        # Tìm các tham chiếu trong điều
        references = self._extract_references(article_content, article_id)
        
        # Đếm tokens (ước tính theo số từ)
        token_count = len(article_content.split())
        
        if token_count <= self.max_chunk_tokens:
            # Điều ngắn: giữ nguyên toàn bộ
            metadata = ChunkMetadata(
                article_id=article_id,
                title=title,
                type="article",
                references=references
            )
            
            return [LegalChunk(
                text=article_title + " " + article_content,
                metadata=metadata
            )]
        else:
            # Điều dài: phân theo khoản
            return self._split_by_paragraphs(
                article_title, article_id, title, article_content, references
            )
    
    def _split_by_paragraphs(
        self, article_title, article_id, title, article_content, references
    ) -> List[LegalChunk]:
        """Phân tách điều thành các chunk theo khoản."""
        chunks = []
        
        # Tách khoản
        paragraphs = re.split(r'(\d+\.)', article_content)
        current_chunk_text = article_title + " " + title
        current_paragraphs = []
        current_refs = []
        
        for j in range(1, len(paragraphs), 2):
            if j+1 < len(paragraphs):
                para_num = paragraphs[j].strip().replace(".", "")
                para_content = paragraphs[j+1].strip()
                
                # Tìm tham chiếu trong khoản
                para_refs = self._extract_references(para_content, article_id)
                
                # Kiểm tra độ dài sau khi thêm khoản mới
                candidate = current_chunk_text + " " + para_num + ". " + para_content
                candidate_tokens = len(candidate.split())
                
                if candidate_tokens <= self.max_chunk_tokens:
                    # Thêm khoản vào chunk hiện tại
                    current_chunk_text = candidate
                    current_paragraphs.append(para_num)
                    current_refs.extend(para_refs)
                else:
                    # Lưu chunk hiện tại
                    metadata = ChunkMetadata(
                        article_id=article_id,
                        title=title,
                        type="article_part",
                        paragraphs=current_paragraphs.copy(),
                        references=current_refs.copy()
                    )
                    
                    chunks.append(LegalChunk(
                        text=current_chunk_text,
                        metadata=metadata
                    ))
                    
                    # Bắt đầu chunk mới với bối cảnh
                    # Thêm tiêu đề điều và khoản này
                    current_chunk_text = f"{article_title} (tiếp) {para_num}. {para_content}"
                    current_paragraphs = [para_num]
                    current_refs = para_refs.copy()
        
        # Thêm chunk cuối cùng
        if current_paragraphs:
            metadata = ChunkMetadata(
                article_id=article_id,
                title=title,
                type="article_part",
                paragraphs=current_paragraphs,
                references=current_refs
            )
            
            chunks.append(LegalChunk(
                text=current_chunk_text,
                metadata=metadata
            ))
        
        return chunks
    
    def _extract_references(self, text: str, current_article_id: str) -> List[LegalReference]:
        """Trích xuất tham chiếu từ văn bản."""
        references = []
        
        # Tìm tham chiếu đến điều khác
        for match in self.reference_patterns["other_article"].finditer(text):
            ref = LegalReference(
                article=match.group(6),
                paragraph=match.group(2) if match.group(1).lower() == "khoản" else None,
                point=match.group(2) if match.group(1).lower() == "điểm" else None,
                raw_text=match.group(0)
            )
            references.append(ref)
        
        # Tìm tham chiếu trong cùng điều
        for match in self.reference_patterns["same_article"].finditer(text):
            ref = LegalReference(
                article=current_article_id,
                paragraph=match.group(2) if match.group(1).lower() == "khoản" else None,
                point=match.group(2) if match.group(1).lower() == "điểm" else None,
                is_current_article=True,
                raw_text=match.group(0)
            )
            references.append(ref)
        
        # Tìm tham chiếu chỉ đến điều
        for match in self.reference_patterns["article_only"].finditer(text):
            article_id = match.group(1)
            # Tránh trùng lặp với các điều đã tìm thấy
            if not any(r.article == article_id and r.paragraph is None and r.point is None for r in references):
                ref = LegalReference(
                    article=article_id,
                    raw_text=match.group(0)
                )
                references.append(ref)
        
        return references

class LegalEmbedding:
    """Tạo và quản lý embedding cho tài liệu pháp lý."""
    
    def __init__(self, model_name='paraphrase-multilingual-MiniLM-L12-v2'):
        self.model = SentenceTransformer(model_name, trust_remote_code=True)
    
    def create_embeddings(self, chunks: List[LegalChunk], method='enhanced') -> List[LegalChunk]:
        """Tạo embedding cho các chunk theo phương pháp chỉ định."""
        if method == 'basic':
            return self._create_basic_embeddings(chunks)
        elif method == 'enhanced':
            return self._create_enhanced_embeddings(chunks)
        elif method == 'hierarchical':
            return self._create_hierarchical_embeddings(chunks)
        else:
            raise ValueError(f"Phương pháp embedding không hợp lệ: {method}")
    
    def _create_basic_embeddings(self, chunks: List[LegalChunk]) -> List[LegalChunk]:
        """Tạo embedding cơ bản cho các chunk."""
        texts = [chunk.text for chunk in chunks]
        embeddings = self.model.encode(texts)
        
        for i, chunk in enumerate(chunks):
            chunk.embedding = embeddings[i]
        
        return chunks
    
    def _create_enhanced_embeddings(self, chunks: List[LegalChunk]) -> List[LegalChunk]:
        """Tạo embedding nâng cao với tính năng pháp lý."""
        
        for chunk in chunks:
            # 1. Tạo văn bản nâng cao với trọng số cho các phần quan trọng
            enhanced_text = chunk.text
            
            # Tăng cường tiêu đề điều và số điều
            article_title_match = re.search(r'(Điều \d+\.\s*[^\.\']+)', enhanced_text)
            if article_title_match:
                article_title = article_title_match.group(1)
                # Lặp lại tiêu đề 2 lần để tăng trọng số trong embedding
                enhanced_text = article_title + " " + article_title + " " + enhanced_text
            
            # Tăng cường số khoản, điểm
            para_points = re.findall(r'(\d+\.\s*[^\.\']+|[a-z]\)\s*[^;]+)', enhanced_text)
            if para_points:
                # Thêm các khoản, điểm vào đầu văn bản để tăng trọng số
                para_points_text = " ".join(para_points[:3])  # Giới hạn 3 khoản/điểm đầu tiên
                enhanced_text = para_points_text + " " + enhanced_text
            
            # 2. Tạo embedding cho văn bản đã tăng cường
            chunk.embedding = self.model.encode(enhanced_text)
        
        return chunks
    
    def _create_hierarchical_embeddings(self, chunks: List[LegalChunk]) -> List[LegalChunk]:
        """Tạo embedding phân cấp cho các chunk."""
        
        # Tổ chức chunks theo Điều
        article_dict = {}
        for chunk in chunks:
            article_id = chunk.metadata.article_id
            if article_id not in article_dict:
                article_dict[article_id] = []
            article_dict[article_id].append(chunk)
        
        # Tạo embedding cấp Điều
        article_embeddings = {}
        for article_id, article_chunks in article_dict.items():
            # Ghép tất cả chunk của điều này
            full_article_text = " ".join([chunk.text for chunk in article_chunks])
            
            # Tạo embedding cấp Điều
            article_embeddings[article_id] = self.model.encode(full_article_text)
        
        # Kết hợp embedding cấp Điều với embedding cấp chunk
        for chunk in chunks:
            # Tạo embedding cấp chunk
            chunk_embedding = self.model.encode(chunk.text)
            
            # Kết hợp với embedding cấp Điều (với trọng số)
            article_embedding = article_embeddings[chunk.metadata.article_id]
            combined_embedding = 0.7 * chunk_embedding + 0.3 * article_embedding
            
            # Chuẩn hóa
            norm = np.linalg.norm(combined_embedding)
            if norm > 0:
                combined_embedding = combined_embedding / norm
            
            chunk.embedding = combined_embedding
        
        return chunks

    def search(self, query: str, chunks: List[LegalChunk], top_k: int = 3) -> List[Dict]:
        """Tìm kiếm các chunk liên quan đến truy vấn."""
        query_embedding = self.model.encode(query)
        
        results = []
        for chunk in chunks:
            if chunk.embedding is not None:
                # Tính độ tương đồng cosine
                similarity = np.dot(query_embedding, chunk.embedding) / (
                    np.linalg.norm(query_embedding) * np.linalg.norm(chunk.embedding)
                )
                
                results.append({
                    "chunk": chunk,
                    "similarity": float(similarity)
                })
        
        # Sắp xếp theo độ tương đồng
        results.sort(key=lambda x: x["similarity"], reverse=True)
        
        # Chuyển đổi kết quả sang định dạng dễ sử dụng
        formatted_results = []
        for result in results[:top_k]:
            chunk = result["chunk"]
            formatted_results.append({
                "text": chunk.text,
                "metadata": asdict(chunk.metadata),
                "similarity": result["similarity"]
            })
        
        return formatted_results

## Configuration Constants

In [30]:
# Weaviate configuration
WEAVIATE_URL="https://ekg9amszqaap6mlmw9fixq.c0.us-west3.gcp.weaviate.cloud"
WEAVIATE_API_KEY="WZoR5excldEQt0XqH4E2X3zMEvVfvFzr7lc5"
DATA_COLLECTION = "ND168"

# Device and model configuration
DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"
MODEL_NAME = "dangvantuan/vietnamese-document-embedding"


# Chunking configuration
MAX_CHUNK_TOKENS = 512  # Optimized for Vietnamese legal text
CHUNK_OVERLAP = 50  # Small overlap to maintain context
EMBEDDING_METHOD = "enhanced"  # Options: basic, enhanced, hierarchical

## Load and Process Documents

In [31]:
# Load raw documents
with open("/home/ltnga/LawVN-Instructction-Gen/src/data/data.json") as f:
    data = json.load(f)

# Process each document with specialized legal document processing
legal_processor = LegalDocument(max_chunk_tokens=MAX_CHUNK_TOKENS, overlap_tokens=CHUNK_OVERLAP)
all_chunks = []

for doc_text in data:
    # Process with legal document chunking
    chunks = legal_processor.process(doc_text)
    all_chunks.extend(chunks)

## Create Embeddings

In [32]:
# Initialize embedding generator
embedder = LegalEmbedding(model_name=MODEL_NAME)

# Generate embeddings with the specified method
chunks_with_embeddings = embedder.create_embeddings(all_chunks, method=EMBEDDING_METHOD)

# Optional: Apply Vietnamese tokenization for improved search
for chunk in chunks_with_embeddings:
    # Store original text for display
    chunk.metadata.original_text = chunk.text
    # Tokenize for better vectorization (Vietnamese-specific)
    chunk.tokenized_text = ViTokenizer.tokenize(chunk.text.lower())

## Setup Weaviate Vector Store

In [33]:
# Connect to Weaviate
client = weaviate.connect_to_weaviate_cloud(
    cluster_url=WEAVIATE_URL,
    auth_credentials=Auth.api_key(WEAVIATE_API_KEY),
)

# Check if collection exists, delete if it does
if client.collections.exists(DATA_COLLECTION):
    client.collections.delete(DATA_COLLECTION)

# Create collection with proper schema
collection = client.collections.create(
    name=DATA_COLLECTION,
    vectorizer_config=None,  # We'll provide our own vectors
    properties=[
        {"name": "text", "data_type": "text"},
        {"name": "article_id", "data_type": "text"},
        {"name": "title", "data_type": "text"},
        {"name": "type", "data_type": "text"},
        {"name": "paragraphs", "data_type": "text[]"}
    ]
)

/home/ltnga/LawVN-Instructction-Gen/venv/lib/python3.10/site-packages/weaviate/warnings.py:314: ResourceWarning: Con004: The connection to Weaviate was not closed properly. This can lead to memory leaks.
            Please make sure to close the connection using `client.close()`.
  warnings.warn(


WeaviateInvalidInputError: Invalid input provided: Invalid collection config create parameters: 5 validation errors for _CollectionConfigCreate
properties.0.data_type
  Input should be an instance of DataType [type=is_instance_of, input_value='text', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/is_instance_of
properties.1.data_type
  Input should be an instance of DataType [type=is_instance_of, input_value='text', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/is_instance_of
properties.2.data_type
  Input should be an instance of DataType [type=is_instance_of, input_value='text', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/is_instance_of
properties.3.data_type
  Input should be an instance of DataType [type=is_instance_of, input_value='text', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/is_instance_of
properties.4.data_type
  Inp

## Index Documents in Weaviate

In [ ]:
# Prepare batch import
with collection.batch.dynamic() as batch:
    for i, chunk in enumerate(chunks_with_embeddings):
        # Convert paragraphs list to strings for Weaviate compatibility
        paragraphs_str = [str(p) for p in chunk.metadata.paragraphs]
        
        # Add object with its embedding
        batch.add_object(
            properties={
                "text": chunk.text,
                "article_id": chunk.metadata.article_id,
                "title": chunk.metadata.title,
                "type": chunk.metadata.type,
                "paragraphs": paragraphs_str
            },
            vector=chunk.embedding.tolist() if chunk.embedding is not None else None
        )

## Create Query Interface

In [ ]:
def search_legal_documents(query, top_k=5):
    """Search for legal documents matching the query."""
    # Tokenize query for Vietnamese
    tokenized_query = ViTokenizer.tokenize(query.lower())
    
    # Create query embedding using our legal embedder
    query_embedding = embedder.model.encode(tokenized_query)
    
    # Search in Weaviate
    results = collection.query.near_vector(
        near_vector=query_embedding.tolist(),
        limit=top_k,
        return_properties=["text", "article_id", "title", "type", "paragraphs"]
    )
    
    return results.objects

## Test Search

In [ ]:
# Test the search
query = "Không đội mũ bảo hiểm thì bị phạt bao nhiêu tiền?"
results = search_legal_documents(query)

for i, result in enumerate(results):
    print(f"Result {i+1}:")
    print(f"Article ID: {result.properties['article_id']}")
    
    # Extract and display the relevant part about helmets
    text = result.properties["text"]
    if "mũ bảo hiểm" in text.lower():
        # Display the article headline
        headline = text.split("\n")[0] if "\n" in text else text[:100]
        print(f"Text: {headline}")
        
        # Find and display paragraphs containing the helmet info
        for paragraph in text.split("\n"):
            if "mũ bảo hiểm" in paragraph.lower():
                print(f"...\n{paragraph}")
    else:
        # Just show a preview if helmet info isn't explicitly mentioned
        print(f"Text: {text[:200]}...")
    
    print("\n" + "=" * 48)

Result 1:
Article ID: 16
Text: Điều 16. Xử phạt, trừ điểm giấy phép lái xe của người điều khiển xe mô tô, xe gắn máy (kể cả xe máy điện), các loại xe tương tự xe mô tô và các loại xe tương tự xe gắn máy vi phạm quy tắc giao thông đường bộ
...
h) Không đội "mũ bảo hiểm cho người đi mô tô, xe máy" hoặc đội "mũ bảo hiểm cho người đi mô tô, xe máy" không cài quai đúng quy cách khi điều khiển xe tham gia giao thông trên đường bộ;
i) Chở người ngồi trên xe không đội "mũ bảo hiểm cho người đi mô tô, xe máy" hoặc đội "mũ bảo hiểm cho người đi mô tô, xe máy" không cài quai đúng quy cách, trừ trường hợp chở người bệnh đi cấp cứu, trẻ em dưới 06 tuổi, áp giải người có hành vi vi phạm pháp luật;

Result 2:
...


## Advanced Search With Specific Reference Extraction

In [ ]:
def extract_penalty_info(query, top_k=3):
    """Extract specific penalty information based on the query."""
    results = search_legal_documents(query, top_k=top_k)
    
    penalties = []
    for result in results:
        text = result.properties["text"]
        
        # Look for penalty amounts
        money_patterns = re.findall(r'phạt tiền từ ([\d\.]+) đồng đến ([\d\.]+) đồng', text, re.IGNORECASE)
        
        # Extract relevant behavior descriptions
        behaviors = []
        for line in text.split("\n"):
            if "mũ bảo hiểm" in line.lower() and ("không đội" in line.lower() or "không cài quai" in line.lower()):
                behaviors.append(line.strip())
        
        # If we found both penalty and behavior
        if money_patterns and behaviors:
            for min_amount, max_amount in money_patterns:
                penalties.append({
                    "article_id": result.properties["article_id"],
                    "min_amount": min_amount,
                    "max_amount": max_amount,
                    "behaviors": behaviors,
                    "excerpt": text[:300] + "..." if len(text) > 300 else text
                })
    
    return penalties

In [ ]:
# Extract specific penalty information
query = "Không đội mũ bảo hiểm khi đi xe máy mức phạt"
penalty_info = extract_penalty_info(query)

# Format the results
print("=== MỨC PHẠT TIỀN KHI KHÔNG ĐỘI MŨ BẢO HIỂM ===")
print()

for penalty in penalty_info:
    print(f"Theo Điều {penalty['article_id']}:")
    print(f"- Mức phạt: {penalty['min_amount']} đồng đến {penalty['max_amount']} đồng")
    print("- Áp dụng cho hành vi:")
    for behavior in penalty['behaviors']:
        print(f"  * {behavior}")

=== MỨC PHẠT TIỀN KHI KHÔNG ĐỘI MŨ BẢO HIỂM ===

Theo Điều 16:
- Mức phạt: 400.000 đồng đến 600.000 đồng
- Áp dụng cho hành vi:
  * Không đội "mũ bảo hiểm cho người đi mô tô, xe máy" hoặc đội "mũ bảo hiểm cho người đi mô tô, xe máy" không cài quai đúng quy cách khi điều khiển xe tham gia giao thông trên đường bộ
  * Chở người ngồi trên xe không đội "mũ bảo hiểm cho người đi mô tô, xe máy" hoặc đội "mũ bảo hiểm cho người đi mô tô, xe máy" không cài quai đúng quy cách, trừ trường hợp chở người bệnh đi cấp cứu, trẻ em dưới 06 tuổi, áp giải người có hành vi vi phạm pháp luật
